# Run C Code

In [34]:
import os
import numpy as np
import scipy
#from scipy.stats import halfcauchy 

## Input Data

In [35]:
# dimension
n = 1000
p = 50

sigma_eps = 1.3

# para
np.random.seed(456)
beta_gt = np.random.uniform(-5, 5, size=p).round(1)
beta_gt[np.random.binomial(n=1, p=0.5, size=p)==1]=0

# data
X = np.random.normal(loc=0, scale=2, size=n*p).reshape(n,-1)
y = np.random.normal(X@beta_gt, scale=sigma_eps)

In [36]:
def HorseShoe_Gibbs(n_sim:int=100, burnin:int=0, thinning:int=1):
    """
    Run Horshoe Gibbs sampler
    """
    
    # D0
    n, p = X.shape

    # setup values
    alphas_out = np.zeros((p, 1))
    s2_out     = np.zeros((1, 1))
    t2_out     = np.zeros((1, 1))
    l2_out     = np.zeros((p, 1))
    
    # sample priors
    betas   = scipy.stats.halfcauchy.rvs(size=p)
    tau_2   = scipy.stats.halfcauchy.rvs(size=1)                            
    nu      = np.ones(p) 
    
    sigma_2, xi = 1.0, 1.0
    
    # Gibbs sampler: run MCMC
    for k in range(n_sim):
        sigma = np.sqrt(sigma_2)

        # alphas
        # - Sigma_star
        Sigma_star     = tau_2 * np.diag(betas**2) # Sigma_star
        Sigma_star_inv = np.diag(1. / betas**2) * (1. / tau_2)
        
        # compute matrices A, L
        A = X.T @ X + Sigma_star_inv
        L = np.linalg.cholesky(A)

        # compute conditional mean
        cond_mu = np.linalg.solve(L.T, np.linalg.solve(L, X.T @ y))

        # compute conditional covariance matrix
        cond_cov = sigma_2 * np.linalg.inv(L.T) @ np.linalg.inv(L)
        
        # sample alphas from conditional Gaussian dist.
        alphas = np.random.multivariate_normal(mean=cond_mu, cov=cond_cov, size=1).T[:,0]

        # sigma_2
        sigma_2 = scipy.stats.invgamma.rvs(0.5*(n + p), scale=0.5*(np.linalg.norm((y - X @ alphas), 2)**2 + (alphas.T @ Sigma_star_inv @ alphas)))

        # - betas
        betas = np.sqrt(scipy.stats.invgamma.rvs(np.ones(p), scale=(1. / nu) + (alphas**2)/(2 * sigma_2 * tau_2)))

        # - tau_2
        tau_2 = scipy.stats.invgamma.rvs(0.5*(p + 1), scale=1.0 / xi + (1. / (2. * sigma_2)) * sum(alphas**2 / betas**2), size=1)

        # - nu
        nu = scipy.stats.invgamma.rvs(np.ones(p), scale=1.0 + betas**(-2), size=p)

        # - xi
        xi = scipy.stats.invgamma.rvs(1.0, scale=1.0 + 1.0 / tau_2, size=1)
        
        # store samples
        if k > burnin:
            # - append
            if(k%thinning==0):
                alphas_out = np.append(arr=alphas_out, values=alphas.reshape(-1,1), axis=1)
                s2_out = np.append(s2_out, sigma_2)
                t2_out = np.append(t2_out, tau_2)
                l2_out = np.append(arr=l2_out, values=betas.reshape(-1,1), axis=1)
    
    return alphas

In [37]:
HorseShoe_Gibbs(n_sim=1000, burnin=100, thinning=2).round(1)

array([-2.5, -3.4,  2.8,  3. ,  0. , -0.1, -0.1, -0. ,  0. , -0. ,  0. ,
       -1.1,  0.8, -3.5,  0. , -0.3,  0.7, -0. ,  0. ,  1.8,  0. , -3.8,
       -0. , -4.9, -1.4, -3.5,  0.4,  3.6, -2.9,  0. , -0. ,  4.7,  0. ,
        0. , -0.7, -0. , -0. , -0. , -0. , -0. , -0. , -0. , -0. ,  4. ,
       -3.2,  0.9,  0. ,  1.5,  0. ,  0. ])

In [32]:
beta_gt

array([-2.5, -3.4,  2.8,  3.1,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
       -1.1,  0.8, -3.5,  0. , -0.3,  0.7,  0. ,  0. ,  1.8,  0. , -3.8,
        0. , -4.9, -1.4, -3.5,  0.4,  3.6, -3. ,  0. ,  0. ,  4.7,  0. ,
        0. , -0.7,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  4. ,
       -3.2,  0.9,  0. ,  1.5,  0. ,  0. ])